tgb - 4/15/2021 - Recycling this notebook but fitting in percentile space (no scale_dict, use output in percentile units)

tgb - 4/15/2020
- Adapting Ankitesh's notebook that builds and train a "brute-force" network to David Walling's hyperparameter search  
- Adding the option to choose between aquaplanet and real-geography data

In [1]:
import sys
sys.path.insert(1,"/home1/07064/tg863631/anaconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages") #work around for h5py
from cbrain.imports import *
from cbrain.cam_constants import *
from cbrain.utils import *
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
from cbrain.climate_invariant import *

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow_probability as tfp
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
# import cartopy.crs as ccrs
import matplotlib.ticker as mticker
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
# from climate_invariant import *
from tensorflow.keras import layers
import datetime
from climate_invariant_utils import *
import yaml


/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


## Global Variables

In [2]:
# Load coordinates (just pick any file from the climate model run)

# Comet path below
# coor = xr.open_dataset("/oasis/scratch/comet/ankitesh/temp_project/data/sp8fbp_minus4k.cam2.h1.0000-01-01-00000.nc",\
#                     decode_times=False)

# GP path below
path_0K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/'
coor = xr.open_dataset(path_0K+"AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-09-02-00000.nc")

lat = coor.lat; lon = coor.lon; lev = coor.lev;
coor.close();

In [3]:
# Comet path below
# TRAINDIR = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/CRHData/'
# path = '/home/ankitesh/CBrain_project/CBRAIN-CAM/cbrain/'

# GP path below
TRAINDIR = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path = '/export/nfs0home/tbeucler/CBRAIN-CAM/cbrain/'
path_nnconfig = '/export/nfs0home/tbeucler/CBRAIN-CAM/nn_config/'

# Load hyam and hybm to calculate pressure field in SPCAM
path_hyam = 'hyam_hybm.pkl'
hf = open(path+path_hyam,'rb')
hyam,hybm = pickle.load(hf)

# Scale dictionary to convert the loss to W/m2
scale_dict = load_pickle(path_nnconfig+'scale_dicts/009_Wm2_scaling.pkl')

New Data generator class for the climate-invariant network. Calculates the physical rescalings needed to make the NN climate-invariant

## Data Generators

### Choose between aquaplanet and realistic geography here

In [4]:
# GP paths below
#path_aquaplanet = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
#path_realgeography = ''

# GP /fast paths below
path_aquaplanet = '/fast/tbeucler/climate_invariant/aquaplanet/'

# Comet paths below
# path_aquaplanet = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/'
# path_realgeography = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/geography/'

path = path_aquaplanet

### Data Generator using RH

In [5]:
#scale_dict_RH = load_pickle('/home/ankitesh/CBrain_project/CBRAIN-CAM/nn_config/scale_dicts/009_Wm2_scaling_2.pkl')
scale_dict_RH = scale_dict.copy()
scale_dict_RH['RH'] = 0.01*L_S/G, # Arbitrary 0.1 factor as specific humidity is generally below 2%

in_vars_RH = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_realgeography: out_vars_RH = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
# elif path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
if path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','QRL','QRS']

# New GP path below
TRAINFILE_RH = '2021_01_24_O3_small_shuffle.nc'
NORMFILE_RH = '2021_02_01_NORM_O3_RH_small.nc'
    
# Comet/Ankitesh path below
# TRAINFILE_RH = 'CI_RH_M4K_NORM_train_shuffle.nc'
# NORMFILE_RH = 'CI_RH_M4K_NORM_norm.nc'
# VALIDFILE_RH = 'CI_RH_M4K_NORM_valid.nc'

In [6]:
train_gen_RH = DataGenerator(
    data_fn = path+TRAINFILE_RH,
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = path+NORMFILE_RH,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict_RH,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using QSATdeficit

We only need the norm file for this generator as we are solely using it as an input to determine the right normalization for the combined generator

In [7]:
# New GP path below
TRAINFILE_QSATdeficit = '2021_02_01_O3_QSATdeficit_small_shuffle.nc'
NORMFILE_QSATdeficit = '2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [8]:
in_vars_QSATdeficit = ['QSATdeficit','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
# if path==path_realgeography: out_vars_RH = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']
# elif path==path_aquaplanet: out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
if path==path_aquaplanet: out_vars_QSATdeficit = ['PHQ','TPHYSTND','QRL','QRS']

In [9]:
train_gen_QSATdeficit = DataGenerator(
    data_fn = path+TRAINFILE_QSATdeficit,
    input_vars = in_vars_QSATdeficit,
    output_vars = out_vars_QSATdeficit,
    norm_fn = path+NORMFILE_QSATdeficit,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using TNS

In [10]:
in_vars = ['QBP','TfromNS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_TNS = '2021_02_01_O3_TfromNS_small_shuffle.nc'
NORMFILE_TNS = '2021_02_01_NORM_O3_TfromNS_small.nc'
VALIDFILE_TNS = 'CI_TNS_M4K_NORM_valid.nc'

In [11]:
train_gen_TNS = DataGenerator(
    data_fn = path+TRAINFILE_TNS,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_TNS,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using BCONS

In [12]:
in_vars = ['QBP','BCONS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_BCONS = '2021_02_01_O3_BCONS_small_shuffle.nc'
NORMFILE_BCONS = '2021_02_01_NORM_O3_BCONS_small.nc'

In [13]:
train_gen_BCONS = DataGenerator(
    data_fn = path+TRAINFILE_BCONS,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_BCONS,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using BMSE

In [14]:
in_vars = ['QBP','BMSE','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_BMSE = '2021_06_16_BMSE_small_shuffle.nc'
NORMFILE_BMSE = '2021_06_16_NORM_BMSE_small.nc'

In [15]:
train_gen_BMSE = DataGenerator(
    data_fn = path+TRAINFILE_BMSE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_BMSE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
)

### Data Generator using NSto220

In [16]:
in_vars = ['QBP','T_NSto220','PS', 'SOLIN', 'SHFLX', 'LHFLX']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_T_NSto220 = '2021_03_31_O3_T_NSto220_small.nc'
NORMFILE_T_NSto220 = '2021_03_31_NORM_O3_T_NSto220_small.nc'

In [17]:
train_gen_T_NSto220 = DataGenerator(
    data_fn = path+TRAINFILE_T_NSto220,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_T_NSto220,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator using LHF_nsDELQ

In [18]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHF_nsDELQ']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_LHF_nsDELQ = '2021_02_01_O3_LHF_nsDELQ_small_shuffle.nc'
NORMFILE_LHF_nsDELQ = '2021_02_01_NORM_O3_LHF_nsDELQ_small.nc'

In [19]:
train_gen_LHF_nsDELQ = DataGenerator(
    data_fn = path+TRAINFILE_LHF_nsDELQ,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_LHF_nsDELQ,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator using LHF_nsQ

In [20]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHF_nsQ']
if path==path_aquaplanet: out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']
elif path==path_realgeography: out_vars = ['PTEQ','PTTEND','FSNT','FSNS','FLNT','FLNS']

TRAINFILE_LHF_nsQ = '2021_02_01_O3_LHF_nsQ_small_shuffle.nc'
NORMFILE_LHF_nsQ = '2021_02_01_NORM_O3_LHF_nsQ_small.nc'

In [21]:
train_gen_LHF_nsQ = DataGenerator(
    data_fn = path+TRAINFILE_LHF_nsQ,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path+NORMFILE_LHF_nsQ,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=8192,
    shuffle=True,
)

### Data Generator Combined (latest flexible version)

In [22]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
#if path==path_aquaplanet: out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']
out_vars = ['PHQ','TPHYSTND','QRL','QRS']

In [23]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In physical space
TRAINFILE = '2021_03_18_O3_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_03_18_O3_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_03_18_O3_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_18_RG_TRAIN_M4K_shuffle.nc'

# In percentile space
#TRAINFILE = '2021_04_09_PERC_TRAIN_M4K_shuffle.nc'
#TRAINFILE = '2021_01_24_O3_small_shuffle.nc'
#VALIDFILE = '2021_04_09_PERC_VALID_M4K.nc'
#TESTFILE = '2021_04_09_PERC_TEST_P4K.nc'

Old data generator by Ankitesh

Improved flexible data generator

In [24]:
train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [25]:
valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [26]:
test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [27]:
test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

## Add callback class to track loss on multiple sets during training

From [https://stackoverflow.com/questions/47731935/using-multiple-validation-sets-with-keras]

In [ ]:
test_diffgeog_gen_CI[0][0].shape

In [ ]:
np.argwhere(np.isnan(test_gen_CI[0][1]))

In [ ]:
np.argwhere(np.isnan(test_gen_CI[0][0]))

In [28]:
class AdditionalValidationSets(Callback):
    def __init__(self, validation_sets, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            valid_generator,valid_name = validation_set
            #tf.print('Results')
            results = self.model.evaluate_generator(generator=valid_generator)
            #tf.print(results)

            for metric, result in zip(self.model.metrics_names,[results]):
                #tf.print(metric,result)
                valuename = valid_name + '_' + metric
                self.history.setdefault(valuename, []).append(result)

## Models tracking losses across climates and geography (Based on cold Aquaplanet)

### MLR or Logistic regression

#### BF

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_MLR'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
#model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=T-TNS)

In [ ]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_MLR_RH_TfromNS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_MLR_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BMSE but no LHF rescaling)

In [70]:
Tscaling_name = 'BMSE'
train_gen_T = train_gen_BMSE

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [71]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [72]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_26 (Dense)             (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [74]:
Tscaling_name = 'BMSE'
train_gen_T = train_gen_BMSE

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [75]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [76]:
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_27 (Dense)             (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [78]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_06_19_MLR_RH_BMSE'

In [79]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [80]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 4302s 747ms/step - loss: 334.5039 - val_loss: 319.9624
Epoch 2/20
4107/5759 [====================>.........] - ETA: 3:28 - loss: 312.7469

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BMSE)

In [35]:
Tscaling_name = 'BMSE'
train_gen_T = train_gen_BMSE

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [36]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [37]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 120)               7800      
Total params: 7,800
Trainable params: 7,800
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [39]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_06_17_MLR_RH_BMSE_LHF_nsDELQ'

In [40]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [41]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [42]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 5008s 870ms/step - loss: 334.2580 - val_loss: 319.4431
Epoch 2/20
5759/5759 [==============================] - 5021s 872ms/step - loss: 310.9616 - val_loss: 308.0854
Epoch 3/20
5759/5759 [==============================] - 4962s 862ms/step - loss: 304.2928 - val_loss: 303.6129
Epoch 4/20
5759/5759 [==============================] - 4874s 846ms/step - loss: 301.2343 - val_loss: 301.3579
Epoch 5/20
5759/5759 [==============================] - 4813s 836ms/step - loss: 299.4814 - val_loss: 300.0119
Epoch 6/20
5759/5759 [==============================] - 4878s 847ms/step - loss: 298.3330 - val_loss: 299.0905
Epoch 7/20
5759/5759 [==============================] - 5191s 901ms/step - loss: 297.4831 - val_loss: 298.3962
Epoch 8/20
5759/5759 [==============================] - 4901s 851ms/step - loss: 296.8217 - val_loss: 297.8356
Epoch 9/20
5759/5759 [==============================] - 4916s 854ms/step - loss: 296.2933 - val_loss: 297.4064
E

In [43]:
hist_rec = history.history

In [44]:
hist_rec

{'loss': [334.25796028159726,
  310.96164104172203,
  304.29278532658464,
  301.2343294329477,
  299.48144232645114,
  298.33301015205734,
  297.48310824760364,
  296.8217336313864,
  296.2932858302333,
  295.86236145952176,
  295.50480239584664,
  295.20159655897373,
  294.9393881234594,
  294.71054514325067,
  294.50793320314193,
  294.3273555187948,
  294.1652634586388,
  294.01803890121624,
  293.8839282250938,
  293.76082433166346],
 'val_loss': [319.44307463043486,
  308.08540499075053,
  303.6129072119215,
  301.3579043039887,
  300.0119019485259,
  299.0905194897258,
  298.39616377757966,
  297.8355510646401,
  297.4063567572401,
  297.0382512328626,
  296.73724090715757,
  296.4812351222283,
  296.2600548311389,
  296.0746194386147,
  295.9009638364166,
  295.75083450299286,
  295.603857946097,
  295.48110678348223,
  295.3582401265941,
  295.2802404797564],
 'trainP4K_loss': [723.5835125951805,
  701.6375067595455,
  691.2493396672261,
  685.9393711901514,
  682.7936574754087

In [45]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=T-TNS)

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']

In [ ]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In percentile space
TRAINFILE = '2021_04_09_PERC_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_04_09_PERC_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_04_09_PERC_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_24_RG_PERC_TRAIN_M4K_shuffle.nc'

In [ ]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_LOGI_PERC_RH_TfromNS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_LOGI_PERC_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=BMSE)

In [46]:
Tscaling_name = 'BMSE'
train_gen_T = train_gen_BMSE

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [47]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_LOGI_PERC_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### NN

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
#if path==path_aquaplanet: out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']
out_vars = ['PHQ','TPHYSTND','QRL','QRS']

In [ ]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In physical space
TRAINFILE = '2021_03_18_O3_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_03_18_O3_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_03_18_O3_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_18_RG_TRAIN_M4K_shuffle.nc'

In [ ]:
train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [ ]:
valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling=None,
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=None,
                               inp_div_Qscaling=None,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

In [ ]:
test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling=None,
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=None,
                                       inp_div_Qscaling=None,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')



#### BF

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
#model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (only Q to RH)

In [ ]:
train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=None,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=None,
                               inp_div_Tscaling=None,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=None,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=None,
                                       inp_div_Tscaling=None,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_06_13_NN_RH'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=T-TNS)

In [ ]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_RH_TfromNS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BCONS and Q=RH but no LHF rescaling)

In [29]:
Tscaling_name = 'BMSE'
train_gen_T = train_gen_BMSE

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling=None,
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=None,
                               inp_div_LHFscaling=None,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling=None,
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=None,
                                       inp_div_LHFscaling=None,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [30]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [31]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [32]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [33]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_06_21_NN_RH_BMSE'

In [34]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [35]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [36]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 4472s 777ms/step - loss: 202.3056 - val_loss: 189.9146
Epoch 2/20
5759/5759 [==============================] - 4475s 777ms/step - loss: 185.0937 - val_loss: 182.6116
Epoch 3/20
5759/5759 [==============================] - 4828s 838ms/step - loss: 179.9384 - val_loss: 178.7548
Epoch 4/20
5759/5759 [==============================] - 4874s 846ms/step - loss: 177.2520 - val_loss: 177.4287
Epoch 5/20
5759/5759 [==============================] - 4869s 845ms/step - loss: 175.4743 - val_loss: 175.0096
Epoch 6/20
5759/5759 [==============================] - 4962s 862ms/step - loss: 174.2388 - val_loss: 174.3024
Epoch 7/20
5759/5759 [==============================] - 4879s 847ms/step - loss: 173.2723 - val_loss: 173.5433
Epoch 8/20
5759/5759 [==============================] - 4723s 820ms/step - loss: 172.4525 - val_loss: 172.4568
Epoch 9/20
5759/5759 [==============================] - 4627s 803ms/step - loss: 171.7867 - val_loss: 172.5883
E

In [37]:
hist_rec = history.history

In [38]:
hist_rec

{'loss': [202.30555719798545,
  185.0937276404027,
  179.9384251054862,
  177.25202826638875,
  175.47426605886997,
  174.23877292533035,
  173.27233092647023,
  172.4525084802727,
  171.78665433242142,
  171.18700743706862,
  170.69160427246518,
  170.25136943061352,
  169.88791444397236,
  169.5348850600648,
  169.22203094699685,
  168.94214552892416,
  168.69187709982558,
  168.49443344325525,
  168.30593784370032,
  168.13759890048118],
 'val_loss': [189.91458921902628,
  182.61157354774585,
  178.75477953714858,
  177.42867169064343,
  175.00958189656527,
  174.30241403865668,
  173.54330357224097,
  172.4567609396341,
  172.58827982105564,
  171.65948165797911,
  170.8602754908579,
  170.41770546286466,
  170.86248802819657,
  170.933456565654,
  169.8005594508478,
  169.2302860813829,
  169.61122131347656,
  169.51639834634534,
  169.62545346494815,
  168.60123147500806],
 'trainP4K_loss': [456.8700075224055,
  443.0509411082704,
  432.7037082837054,
  449.7111600202536,
  439.5

In [39]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input Rescaling (T=BMSE)

In [48]:
Tscaling_name = 'BMSE'
train_gen_T = train_gen_BMSE

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=scale_dict,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=scale_dict,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [49]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [50]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 128)               0   

In [51]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [52]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_06_17_NN_RH_BMSE_LHF_nsDELQ'

In [53]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [54]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [55]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 5169s 898ms/step - loss: 201.6545 - val_loss: 189.0531
Epoch 2/20
5759/5759 [==============================] - 5829s 1s/step - loss: 184.8350 - val_loss: 183.2978
Epoch 3/20
5759/5759 [==============================] - 5229s 908ms/step - loss: 179.8133 - val_loss: 178.5000
Epoch 4/20
5759/5759 [==============================] - 5275s 916ms/step - loss: 177.1403 - val_loss: 177.1365
Epoch 5/20
5759/5759 [==============================] - 5266s 914ms/step - loss: 175.5002 - val_loss: 175.1941
Epoch 6/20
5759/5759 [==============================] - 5169s 898ms/step - loss: 174.2956 - val_loss: 174.6869
Epoch 7/20
5759/5759 [==============================] - 5209s 905ms/step - loss: 173.2392 - val_loss: 173.5198
Epoch 8/20
5759/5759 [==============================] - 5186s 901ms/step - loss: 172.4000 - val_loss: 173.3747
Epoch 9/20
5759/5759 [==============================] - 5572s 968ms/step - loss: 171.7521 - val_loss: 173.1536
Epoc

In [56]:
hist_rec = history.history

In [57]:
hist_rec

{'loss': [201.6545446031229,
  184.83496572789602,
  179.81333736546193,
  177.14034477773072,
  175.5001721204124,
  174.29555164250385,
  173.23922536339606,
  172.4000305212875,
  171.75205920259663,
  171.18566137553296,
  170.72682441914117,
  170.32873507600874,
  169.91278402067178,
  169.60627024843163,
  169.27601303468873,
  169.00035308239092,
  168.7736111094625,
  168.54504873835808,
  168.3567127268189,
  168.1828623958958],
 'val_loss': [189.0530553239696,
  183.29775695438892,
  178.4999650182632,
  177.13654904476206,
  175.19409297301408,
  174.68693372651154,
  173.5198103594372,
  173.37467931469234,
  173.15362810422138,
  172.04390540390364,
  171.04550911219832,
  171.1609666201698,
  170.37010151512357,
  170.2621904179623,
  171.21402206046085,
  169.4481330004341,
  169.28067597174834,
  169.14020986167654,
  169.53549741563728,
  168.83162261533795],
 'trainP4K_loss': [447.4123222773678,
  442.0529859502772,
  429.8408128248567,
  426.76447530495744,
  427.16

In [58]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=T-TNS)

In [ ]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars=['PHQPERC','TPHYSTNDPERC','QRLPERC','QRSPERC']

In [ ]:
# TRAINFILE = '2021_01_24_O3_TRAIN_shuffle.nc'
NORMFILE = '2021_01_24_NORM_O3_small.nc'
# VALIDFILE = '2021_01_24_O3_VALID.nc'
# GENTESTFILE = 'CI_SP_P4K_valid.nc'

# In percentile space
TRAINFILE = '2021_04_09_PERC_TRAIN_M4K_shuffle.nc'
VALIDFILE = '2021_04_09_PERC_VALID_M4K.nc'
TESTFILE_DIFFCLIMATE = '2021_04_09_PERC_TRAIN_P4K_shuffle.nc'
TESTFILE_DIFFGEOG = '2021_04_24_RG_PERC_TRAIN_M4K_shuffle.nc'

In [ ]:
Tscaling_name = 'TfromNS'
train_gen_T = train_gen_TNS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_PERC_RH_TfromNS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=BCONS)

In [ ]:
Tscaling_name = 'BCONS'
train_gen_T = train_gen_BCONS

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_26_NN_PERC_RH_BCONS_LHF_nsDELQ'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### Input and Output Rescaling (T=BMSE)

In [59]:
Tscaling_name = 'BMSE'
train_gen_T = train_gen_BMSE

train_gen_CI = DataGeneratorCI(data_fn = path+TRAINFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

valid_gen_CI = DataGeneratorCI(data_fn = path+VALIDFILE,
                               input_vars=in_vars,
                               output_vars=out_vars,
                               norm_fn=path+NORMFILE,
                               input_transform=('mean', 'maxrs'),
                               output_transform=None,
                               batch_size=8192,
                               shuffle=True,
                               xarray=False,
                               var_cut_off=None, 
                               Qscaling='RH',
                               Tscaling=Tscaling_name,
                               LHFscaling='LHF_nsDELQ',
                               SHFscaling=None,
                               output_scaling=False,
                               interpolate=False,
                               hyam=hyam,hybm=hybm,
                               inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                               inp_div_Qscaling=train_gen_RH.input_transform.div,
                               inp_sub_Tscaling=train_gen_T.input_transform.sub,
                               inp_div_Tscaling=train_gen_T.input_transform.div,
                               inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                               inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                               inp_sub_SHFscaling=None,
                               inp_div_SHFscaling=None,
                               lev=None, interm_size=40,
                               lower_lim=6,is_continous=True,Tnot=5,
                               epsQ=1e-3,epsT=1,mode='train')

test_diffclimate_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFCLIMATE,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

test_diffgeog_gen_CI = DataGeneratorCI(data_fn = path+TESTFILE_DIFFGEOG,
                                       input_vars=in_vars,
                                       output_vars=out_vars,
                                       norm_fn=path+NORMFILE,
                                       input_transform=('mean', 'maxrs'),
                                       output_transform=None,
                                       batch_size=8192,
                                       shuffle=True,
                                       xarray=False,
                                       var_cut_off=None, 
                                       Qscaling='RH',
                                       Tscaling=Tscaling_name,
                                       LHFscaling='LHF_nsDELQ',
                                       SHFscaling=None,
                                       output_scaling=False,
                                       interpolate=False,
                                       hyam=hyam,hybm=hybm,
                                       inp_sub_Qscaling=train_gen_RH.input_transform.sub,
                                       inp_div_Qscaling=train_gen_RH.input_transform.div,
                                       inp_sub_Tscaling=train_gen_T.input_transform.sub,
                                       inp_div_Tscaling=train_gen_T.input_transform.div,
                                       inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
                                       inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div,
                                       inp_sub_SHFscaling=None,
                                       inp_div_SHFscaling=None,
                                       lev=None, interm_size=40,
                                       lower_lim=6,is_continous=True,Tnot=5,
                                       epsQ=1e-3,epsT=1,mode='train')

In [60]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [61]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 128)               0   

In [62]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [63]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_06_16_NN_PERC_RH_BMSE_LHF_nsDELQ'

In [64]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [65]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [66]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 5161s 896ms/step - loss: 4.7080e-04 - val_loss: 6.9356e-10
Epoch 2/20
5759/5759 [==============================] - 5143s 893ms/step - loss: 6.8564e-10 - val_loss: 6.9356e-10
Epoch 3/20
5759/5759 [==============================] - 5096s 885ms/step - loss: 6.8564e-10 - val_loss: 6.9356e-10
Epoch 4/20
5759/5759 [==============================] - 5593s 971ms/step - loss: 6.8564e-10 - val_loss: 6.9356e-10
Epoch 5/20
5759/5759 [==============================] - 5260s 913ms/step - loss: 6.8564e-10 - val_loss: 6.9356e-10
Epoch 6/20
5759/5759 [==============================] - 5266s 914ms/step - loss: 6.8564e-10 - val_loss: 6.9356e-10
Epoch 7/20
5759/5759 [==============================] - 4945s 859ms/step - loss: 6.8564e-10 - val_loss: 6.9356e-10
Epoch 8/20
5759/5759 [==============================] - 5502s 955ms/step - loss: 6.8564e-10 - val_loss: 6.9356e-10
Epoch 9/20
5759/5759 [==============================] - 5219s 906ms/step - loss:

In [67]:
hist_rec = history.history

In [68]:
hist_rec

{'loss': [0.00047079924784177113,
  6.856438808985792e-10,
  6.856438806576036e-10,
  6.856438809274963e-10,
  6.856438817757306e-10,
  6.856438831155552e-10,
  6.856438815058379e-10,
  6.856438810335256e-10,
  6.856438800407059e-10,
  6.856438817564525e-10,
  6.856438809949695e-10,
  6.856438806576036e-10,
  6.856438813419744e-10,
  6.856438832215845e-10,
  6.856438818239257e-10,
  6.85643881168472e-10,
  6.856438807925499e-10,
  6.856438805708523e-10,
  6.856438801274571e-10,
  6.856438806865207e-10],
 'val_loss': [6.935558145731251e-10,
  6.9355581431922e-10,
  6.935558138584295e-10,
  6.935558120434792e-10,
  6.935558125889047e-10,
  6.935558135857168e-10,
  6.935558137926023e-10,
  6.935558135575052e-10,
  6.935558139618723e-10,
  6.935558140653151e-10,
  6.935558127111552e-10,
  6.935558129086369e-10,
  6.935558147329912e-10,
  6.935558134540623e-10,
  6.935558151937816e-10,
  6.935558145825289e-10,
  6.935558137831984e-10,
  6.935558126829435e-10,
  6.935558123255957e-10,
  6.93

In [69]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### NN including batch normalization, dropout, maybe L1 and L2

#### BF with both batchnormalization and dropout

See [https://www.perfectlyrandom.org/2019/06/24/a-guide-to-keras-functional-api/]

In [29]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = BatchNormalization(axis=1)(densout) # Careful because axis is the axis *preserved* here
densout = Dropout(0.3)(densout)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = BatchNormalization(axis=1)(densout) # Careful because axis is the axis *preserved* here
    densout = Dropout(0.3)(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [30]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512   

In [31]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [32]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_07_31_NN_BN_Dropout30'

In [33]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [34]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [35]:
#model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

Epoch 1/20
5759/5759 [==============================] - 2933s 509ms/step - loss: 239.8731 - val_loss: 216.7516
Epoch 2/20
5759/5759 [==============================] - 2562s 445ms/step - loss: 228.0895 - val_loss: 216.1341
Epoch 3/20
5759/5759 [==============================] - 2711s 471ms/step - loss: 225.7196 - val_loss: 214.3894
Epoch 4/20
5759/5759 [==============================] - 2747s 477ms/step - loss: 224.4248 - val_loss: 209.3496
Epoch 5/20
5759/5759 [==============================] - 2759s 479ms/step - loss: 223.6310 - val_loss: 211.7563
Epoch 6/20
5759/5759 [==============================] - 2865s 498ms/step - loss: 223.0524 - val_loss: 208.3421
Epoch 7/20
5758/5759 [============================>.] - ETA: 0s - loss: 222.5408

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

#### BF with just dropout

See [https://www.perfectlyrandom.org/2019/06/24/a-guide-to-keras-functional-api/]

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = Dropout(0.3)(densout)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = Dropout(0.3)(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_07_31_NN_Dropout30'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_diffclimate_gen_CI,'trainP4K'),(test_diffgeog_gen_CI,'trainM4K_RG')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
#model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

## Models tracking losses across climates/geography (Warm to Cold)

## Brute-Force Model

### Climate-invariant (T,Q,PS,S0,SHF,LHF)->($\dot{T}$,$\dot{q}$,RADFLUX)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(64, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/oasis/scratch/comet/tbeucler/temp_project/CBRAIN_models/'
save_name = 'BF_temp'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_pos])

### Ozone (T,Q,$O_{3}$,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(94,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_O3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_O3, epochs=Nep, validation_data=valid_gen_O3,\
              callbacks=[earlyStopping, mcp_save_pos])

### No Ozone (T,Q,S0,PS,LHF,SHF)$\rightarrow$($\dot{q}$,$\dot{T}$,lw,sw)

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_01_25_noO3'

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
model.load_weights(path_HDF5+save_name+'.hdf5')

In [ ]:

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0, update_freq=1000,embeddings_freq=1)

In [ ]:
# Nep = 15
# model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
#               callbacks=[earlyStopping, mcp_save_pos])

In [ ]:
Nep = 10
model.fit_generator(train_gen_noO3, epochs=Nep, validation_data=valid_gen_noO3,\
              callbacks=[earlyStopping, mcp_save_pos])

### BF linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_15_MLR_PERC'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 15
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
history.history

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BF Logistic version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_15_Log_PERC'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 15
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
history.history

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BF NN version with test loss tracking

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_08_NN6L'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
history.history

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH Logistic version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
# densout = Dense(128, activation='linear')(inp)
# densout = LeakyReLU(alpha=0.3)(densout)
# for i in range (6):
#     densout = Dense(128, activation='linear')(densout)
#     densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(inp)
dense_out = tf.keras.activations.sigmoid(dense_out)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_15_Log_PERC_RH'

In [ ]:
# history = AdditionalValidationSets([(train_gen_CI,valid_gen_CI,test_gen_CI)])
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 15
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
history.history

In [ ]:
hist_rec = history.history

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### QSATdeficit linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_QSATdeficit'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### TfromNS linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_TfromNS'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### BCONS linear version

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_BCONS'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

## Mixed Model

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH_BCONS'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+(T-TNS)

### RH+NSto220

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_31_MLR_RH_NSto220'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+LHF_nsQ

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_19_MLR_RH_LHF_nsQ'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+TfromNS+LHF_nsDELQ NN version with test loss tracking

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(120, activation='linear')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_09_NN7L_RH_TfromNS_LHF_nsDELQ'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 20
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+TfromNS+LHF_nsQ

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_23_MLR_RH_TfromNS_LHF_nsQ'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+BCONS+LHF_nsDELQ

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_03_23_MLR_RH_BCONS_LHF_nsDELQ'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+NSto220+LHF_nsDELQ

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_01_MLR_RH_NSto220_LHF_nsDELQ'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()

### RH+NSto220+LHF_nsQ

In [ ]:
inp = Input(shape=(64,)) ## input after rh and tns transformation
dense_out = Dense(120, activation='linear')(inp)
model = tf.keras.models.Model(inp, dense_out)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

In [ ]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2021_04_03_MLR_RH_NSto220_LHF_nsQ'

In [ ]:
history = AdditionalValidationSets([(test_gen_CI,'testP4K')])

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Nep = 10
model.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos, history])

In [ ]:
hist_rec = history.history

In [ ]:
hist_rec

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+save_name+'_hist.pkl','wb')

F_data = {'hist':hist_rec}

pickle.dump(F_data,hf)
hf.close()